# Importing agents
This notebook provides sample EDSL code for simulating surveys with agent personas imported from [Persona Hub](https://github.com/tencent-ailab/persona-hub), an open-source repository provided by [Hugging Face](https://huggingface.co/datasets/proj-persona/PersonaHub).

[EDSL](https://github.com/expectedparrot/edsl) is an open-source library for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

### Importing the data
Persona Hub: https://github.com/tencent-ailab/persona-hub

In [1]:
from datasets import load_dataset

ds = load_dataset("proj-persona/PersonaHub", "instruction")

### Selecting data
Here we select some personas to use.

In [2]:
num_agents = 10

personas_list = ds['train']['input persona'][:num_agents]
personas_list

['A theater manager or events coordinator interested in understanding the operational aspects, facilities, and programming of performing arts centers, such as the Broward Center.',
 'An urban planner looking to understand the distribution and organization of public services in the Halifax Regional Municipality.',
 "A high school literature teacher looking for supplementary materials to enrich their curriculum and provide students with a deeper understanding of their state's cultural heritage.",
 'A science fiction writer, exploring the use of secret trials as a theme in stories that examine surveillance, privacy, and power dynamics in society.',
 'A financial analyst specializing in Asian markets and wealthy individuals, interested in tracking the investments and philanthropic activities of billionaires like Gerald Chan.',
 'A high school physics teacher who enjoys applying physics concepts to various sports, curious about the effects of water resistance and body positioning in Paralym

### Creating agents
We create an agent in EDSL by passing a dictionary of `traits` (such as a persona) to an `Agent` object.
Here we use a method for generating a set of agents from a list of traits at once (a list of the personas).
Learn more about [designing agents in EDSL](https://docs.expectedparrot.com/en/latest/agents.html).

In [3]:
from edsl import AgentList

agents = AgentList.from_list("persona", personas_list)

### Using agents to answer a survey
Here we construct some questions and administer them to the agents.
EDSL comes with many [common question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from a model.
Learn more about [constructing surveys](https://docs.expectedparrot.com/en/latest/surveys.html) with different question types, logic and rules.

In [4]:
from edsl import QuestionMultipleChoice, QuestionLinearScale, Survey

q1 = QuestionMultipleChoice(
    question_name = "preferred_mode_commute",
    question_text = "What is your preferred mode of commuting to work?",
    question_options = [ "Car", "Public transportation", "Bike", "Walk", "Work from home", "Other" ] 
)

q2 = QuestionLinearScale(
    question_name = "work_from_home",
    question_text = "On a scale from 1 to 5, how easy is it for you to work from home?",
    question_options = [1,2,3,4,5],
    option_labels = {1:"Not at all easy", 5:"Very easy"}
)

survey = Survey([q1, q2])

### Selecting language models
EDSL works with many popular language models that we can select to generate the responses for the agents.

To see a list of all services:

In [5]:
from edsl import Model

Model.services()

['openai',
 'anthropic',
 'deep_infra',
 'google',
 'groq',
 'bedrock',
 'azure',
 'ollama',
 'test',
 'together',
 'mistral']

To see a list of all available models:

In [6]:
# Model.available()

To select models to use with a survey:

In [7]:
from edsl import ModelList, Model

models = ModelList(
    Model(m) for m in ["gpt-3.5-turbo", "gpt-4o"]
)

We run a survey by adding the agents and models, and then calling the `run()` method.
This generates a formatted dataset of `Results`:

In [8]:
results = survey.by(agents).by(models).run()

### Analyzing results
EDSL comes with [built-in methods for analysis](https://docs.expectedparrot.com/en/latest/results.html).
Here we inspect the responses:

In [9]:
results.select("model", "persona", "preferred_mode_commute", "work_from_home").print(format="rich")

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ model         ┃ agent                                               ┃ answer                  ┃ answer          ┃
┃ .model        ┃ .persona                                            ┃ .preferred_mode_commute ┃ .work_from_home ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gpt-3.5-turbo │ A theater manager or events coordinator interested  │ Car                     │ 3               │
│               │ in understanding the operational aspects,           │                         │                 │
│               │ facilities, and programming of performing arts      │                         │                 │
│               │ centers, such as the Broward Center.                │                         │                 │
├───────────────┼─────────────────────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-4o        │ A theater manager or events coordinator interested  │ Car                     │ 2               │
│               │ in understanding the operational aspects,           │                         │                 │
│               │ facilities, and programming of performing arts      │                         │                 │
│               │ centers, such as the Broward Center.                │                         │                 │
├───────────────┼─────────────────────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-3.5-turbo │ An urban planner looking to understand the          │ Public transportation   │ 3               │
│               │ distribution and organization of public services in │                         │                 │
│               │ the Halifax Regional Municipality.                  │                         │                 │
├───────────────┼─────────────────────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-4o        │ An urban planner looking to understand the          │ Public transportation   │ 5               │
│               │ distribution and organization of public services in │                         │                 │
│               │ the Halifax Regional Municipality.                  │                         │                 │
├───────────────┼─────────────────────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-3.5-turbo │ A high school literature teacher looking for        │ Public transportation   │ 3               │
│               │ supplementary materials to enrich their curriculum  │                         │                 │
│               │ and provide students with a deeper understanding of │                         │                 │
│               │ their state's cultural heritage.                    │                         │                 │
├───────────────┼─────────────────────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-4o        │ A high school literature teacher looking for        │ Car                     │ 3               │
│               │ supplementary materials to enrich their curriculum  │                         │                 │
│               │ and provide students with a deeper understanding of │                         │                 │
│               │ their state's cultural heritage.                    │                         │                 │
├───────────────┼─────────────────────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-3.5-turbo │ A science fiction writer, exploring the use of      │ Public transportation   │ 3               │
│               │ secret trials as a theme in stories that examine    │                         │                 │
│               │ surveillance, privacy, and power dynam

### Constructing traits
In comparing model responses it can be convenient to include shortnames for traits in addition to narrative personas--e.g., just the name of the agent's occupation, age, etc. Here we run a question to extract the occupation from each persona in order to store it as a separate `trait` of each agent. We use `Scenario` objects to represent the data (personas) that we add to the question when we run it. [Learn more about using scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html) to parameterize questions with data and context.

In [10]:
from edsl import QuestionExtract

q = QuestionExtract(
    question_name = "occupation",
    question_text = "{{ persona }}",
    answer_template = {"occupation":"artist"} 
)

Creating scenarios for the personas in order to add them to the question:

In [11]:
from edsl import ScenarioList

scenarios = ScenarioList.from_list("persona", personas_list)

In [12]:
results = q.by(scenarios).run()

In [13]:
results.select("persona", "occupation").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ scenario                                               ┃ answer                                                 ┃
┃ .persona                                               ┃ .occupation                                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ A theater manager or events coordinator interested in  │ {'occupation': 'theater manager or events              │
│ understanding the operational aspects, facilities, and │ coordinator'}                                          │
│ programming of performing arts centers, such as the    │                                                        │
│ Broward Center.                                        │                                                        │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ An urban planner looking to understand the             │ {'occupation': 'urban planner'}                        │
│ distribution and organization of public services in    │                                                        │
│ the Halifax Regional Municipality.                     │                                                        │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ A high school literature teacher looking for           │ {'occupation': 'high school literature teacher'}       │
│ supplementary materials to enrich their curriculum and │                                                        │
│ provide students with a deeper understanding of their  │                                                        │
│ state's cultural heritage.                             │                                                        │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ A science fiction writer, exploring the use of secret  │ {'occupation': 'science fiction writer'}               │
│ trials as a theme in stories that examine              │                                                        │
│ surveillance, privacy, and power dynamics in society.  │                                                        │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ A financial analyst specializing in Asian markets and  │ {'occupation': 'financial analyst'}                    │
│ wealthy individuals, interested in tracking the        │                                                        │
│ investments and philanthropic activities of            │                                                        │
│ billionaires like Gerald Chan.                         │                                                        │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ A high school physics teacher who enjoys applying      │ {'occupation': 'high school physics teacher'}          │
│ physics concepts to various sports, curious about the  │                                                        │
│ effects of water resistance and body positioning in    │                                                        │
│ Paralympic swimming.                                   │                                                        │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ A high school physical education teacher seeking to    │ {'occupation': 'physical education teacher'}           │
│ incorporate Paralympic history and achievements into   │                                                        │
│ the curriculum to inspire and educate students about  

### Designing agents
Here we recreate agents with both the personas and occupations as traits:

In [14]:
occupations_list = [o['occupation'] for o in results.select("occupation").to_list()]
occupations_list

['theater manager or events coordinator',
 'urban planner',
 'high school literature teacher',
 'science fiction writer',
 'financial analyst',
 'high school physics teacher',
 'physical education teacher',
 'high school history teacher',
 'cultural anthropologist',
 'high school earth science teacher']

In [15]:
t = list(zip(personas_list, occupations_list))

In [16]:
from edsl import AgentList, Agent

agents = AgentList(
    Agent(traits = {"persona":p, "occupation":o}) for p,o in t
)

### Running a survey
Here we rerun the survey and then filter and sort results by agent traits:

In [17]:
results = survey.by(agents).by(models).run()

In [18]:
(
    results
    .filter("model.model == 'gpt-4o'")
    .sort_by("work_from_home", reverse=True)
    .select("model", "occupation", "preferred_mode_commute", "work_from_home")
    .print(format="rich")
)

┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ model  ┃ agent                                 ┃ answer                  ┃ answer          ┃
┃ .model ┃ .occupation                           ┃ .preferred_mode_commute ┃ .work_from_home ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gpt-4o │ science fiction writer                │ Work from home          │ 5               │
├────────┼───────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-4o │ financial analyst                     │ Work from home          │ 5               │
├────────┼───────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-4o │ urban planner                         │ Bike                    │ 4               │
├────────┼───────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-4o │ cultural anthropologist               │ Public transportation   │ 4               │
├────────┼───────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-4o │ high school literature teacher        │ Car                     │ 3               │
├────────┼───────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-4o │ high school physics teacher           │ Bike                    │ 3               │
├────────┼───────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-4o │ high school history teacher           │ Car                     │ 3               │
├────────┼───────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-4o │ theater manager or events coordinator │ Car                     │ 2               │
├────────┼───────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-4o │ high school earth science teacher     │ Car                     │ 2               │
├────────┼───────────────────────────────────────┼─────────────────────────┼─────────────────┤
│ gpt-4o │ physical education teacher            │ Bike                    │ 1               │
└────────┴───────────────────────────────────────┴─────────────────────────┴─────────────────┘

### Posting to the Coop
The [Coop](https://www.expectedparrot.com/content/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we demonstrate how to post this notebook:

In [19]:
from edsl import Notebook

In [20]:
n = Notebook(path = "import_agents.ipynb")

In [21]:
n.push(description = "Importing agents", visibility = "public")

{'description': 'Importing agents',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/e1fe0b4c-40bb-42cb-832c-cab7d464a6b0',
 'uuid': 'e1fe0b4c-40bb-42cb-832c-cab7d464a6b0',
 'version': '0.1.37.dev1',
 'visibility': 'public'}

To update an object at the Coop:

In [22]:
n = Notebook(path = "import_agents.ipynb")

In [23]:
n.patch(uuid = "d9ae8689-f0e1-4821-ba9a-c9a4d7dcc348", value = n)

{'status': 'success'}